### Import Libraries

In [1]:
import pandas as pd
import numpy as np
import math

import matplotlib.pyplot as plt
%matplotlib inline
from sklearn import metrics

import warnings
warnings.filterwarnings("ignore")

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.preprocessing import MinMaxScaler

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

### Import Data

In [2]:
data = pd.read_csv("tr_code.csv")
data.drop(data.columns[0],axis=1,inplace=True)
data = data[~data['familyGroup'].isin(['NON_PRODUCT','BREAKFAST_DRINK','GIFT_COUPON'])]

### Data Manipulations

In [3]:
quantity_by_individualItems = data.pivot_table(values=['qty'], index = data.key, columns='ItemName', aggfunc='sum').fillna(0)
quantity_by_individualItems.columns = list(map("_".join, quantity_by_individualItems.columns))
final_dataset = quantity_by_individualItems \
.join(data[["key","Channel","saleType","timeOfDay","receiptToPresentTime"]].set_index("key"),on=['key'],how="left").reset_index()

In [4]:
df = final_dataset[final_dataset["Channel"]=="Front Counter"].drop(["saleType","timeOfDay","Channel","key"], axis = 1)
df = df[df['receiptToPresentTime']!='[nan nan]']
df['receiptToPresentTime'] = df['receiptToPresentTime'].astype('float').map(lambda x : round(x,2))

In [5]:
## set the CHAID input parameters
independent_variable_columns = list(df.drop(["receiptToPresentTime"],axis=1).columns)
dep_variable = 'receiptToPresentTime'

In [6]:
X = df[independent_variable_columns]
y = df[dep_variable]

### Model Creation

In [7]:
from sklearn.tree import DecisionTreeRegressor
regressor = DecisionTreeRegressor(random_state = 0)
regressor.fit(X, y)

DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
                      max_leaf_nodes=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      presort=False, random_state=0, splitter='best')